##Installation of the libraries

In [1]:
!pip install -q transformers

In [2]:
!pip install biopython

In [3]:
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

Torch optimization.

##All libraries needed for training

In [4]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Data processing

Getting rid of the sequences which are outside of the threshold (64-128).

Getting the sequence of a given file in the target folder (contains only the files with desired sequences).

In [6]:
file_path = "AF-A0A1D8PD42-F1-model_v4.cif"
file_model = "AF-A0A1D8PD42-F1-model_v4"
pdbl = PDBList()
pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

sequence = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(sequence))

Desired structure doesn't exists
MSSSNTDNQYPKYINDTTPPTITLKEYDNASWASTTCLDHNPIKNQYIVVVMENPNQIVAIIDQQDNMILDILFKNAHDAHSKQEYSTK


Calculating the angles for the given sequence

In [7]:
#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you wan tin the pdb file

Putting angles in a matrix.

In [8]:
phi_angles_list.append(0)
psi_angles_list.append(0)

phi = np.asarray(phi_angles_list,dtype=np.float32)*(np.pi/180)
psi = np.asarray(psi_angles_list,dtype=np.float32)*(np.pi/180)
angles = np.vstack((psi,phi))

Changing sequence for to be used in the Prot-Bert embedding.

In [9]:
seq_example =  ' '.join(sequence)
seq_example

'M S S S N T D N Q Y P K Y I N D T T P P T I T L K E Y D N A S W A S T T C L D H N P I K N Q Y I V V V M E N P N Q I V A I I D Q Q D N M I L D I L F K N A H D A H S K Q E Y S T K'

##Embedding space creation (using Prot-Bert)

In [10]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
seq = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq, return_tensors='pt')

/Users/goudarzimandanagmail.com/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Getting the dimensions of the embedding space.

In [11]:
N , D = output.last_hidden_state.size()[1], output.last_hidden_state.size()[2]
embedding_prot_bert = output.last_hidden_state


NameError: name 'output' is not defined

In [ ]:
print(N)
print(D)

91
1024


##Single-head self unmasked attention layer

In [12]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output

##Encoder with attention and 2 layer FFNN

In [13]:
class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.ReLU(),
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.ReLU(),
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return output


In [19]:
class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.05)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            self.optimizer.zero_grad()
            #self.scheduler.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")

    


In [20]:
bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
output = bert_model(**encoded_input)
embedded_pb = output.last_hidden_state
N, D = embedded_pb.size()[1], embedded_pb.size()[2]
embeddings = embedded_pb
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 200

model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)
model.forward(embeddings)
criterion = nn.MSELoss()
trainer = TransformerTrainer(model, criterion, num_epochs, embedded_pb, angles)
trainer.train()
model

Epoch 1/200, Loss: 6.060880661010742
Epoch 2/200, Loss: 7.077129364013672
Epoch 3/200, Loss: 5.752579689025879
Epoch 4/200, Loss: 5.5234694480896
Epoch 5/200, Loss: 5.337890625
Epoch 6/200, Loss: 5.1875715255737305
Epoch 7/200, Loss: 5.065812587738037
Epoch 8/200, Loss: 4.967188835144043
Epoch 9/200, Loss: 4.887302875518799
Epoch 10/200, Loss: 4.822595596313477
Epoch 11/200, Loss: 4.7701826095581055
Epoch 12/200, Loss: 4.727727890014648
Epoch 13/200, Loss: 4.6933393478393555
Epoch 14/200, Loss: 4.665485382080078
Epoch 15/200, Loss: 4.642923355102539
Epoch 16/200, Loss: 4.624648094177246
Epoch 17/200, Loss: 4.609845161437988
Epoch 18/200, Loss: 4.5978546142578125
Epoch 19/200, Loss: 4.588142395019531
Epoch 20/200, Loss: 4.580275535583496
Epoch 21/200, Loss: 4.5739030838012695
Epoch 22/200, Loss: 4.568741798400879
Epoch 23/200, Loss: 4.564561367034912
Epoch 24/200, Loss: 4.5611748695373535
Epoch 25/200, Loss: 4.558431625366211
Epoch 26/200, Loss: 4.556210041046143
Epoch 27/200, Loss: 4.5

TransformerModel(
  (self_attention): SelfAttention()
  (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (layer_norm2): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
  (feed_forward): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=2, bias=True)
  )
)

In [21]:
angles_tensor = torch.from_numpy(angles.T)
predicted_angles = model.forward(embedded_pb)[:, :len(angles_tensor)]

In [26]:
pred = predicted_angles*(180/np.pi)
torch.save(pred,'predictions/N_A0A1D8PD42-F1_200_05_SGD.pt')

In [23]:
angles_tensor*(180/np.pi)

tensor([[ 179.9299,   98.1640],
        [ 166.9430,   82.5050],
        [ 175.7644,   79.1991],
        [ 153.2986,   83.0741],
        [ 139.6692,   60.0443],
        [ 176.9491,  102.7947],
        [ 164.3176,   85.2867],
        [ 178.5258,   42.9345],
        [ 171.2007,  127.9657],
        [ 174.5847,  155.6947],
        [ 173.7650,  138.0246],
        [ 172.4886,  137.1495],
        [ 179.3246,  146.5627],
        [ 170.6717,  -34.0441],
        [-176.4155, -179.8297],
        [-173.8806,   -4.4963],
        [-169.4366,  -13.8653],
        [ 173.5445,  110.5009],
        [ 175.3131,  148.9942],
        [ 167.3701,  154.4301],
        [ 176.2569,  130.1698],
        [ 170.4209,  157.8414],
        [ 171.9277,  157.4808],
        [ 175.1485,  -35.6816],
        [ 175.1972,  -41.2917],
        [ 173.9927,  -40.0640],
        [-179.1476,  -50.0214],
        [-179.6617,  -27.9610],
        [-177.6897,   -0.5788],
        [ 176.5145,  132.7397],
        [ 179.8077,  -31.3434],
        